<a href="https://colab.research.google.com/github/gongx030/Etv2_pioneer/blob/master/ATAC_analysis_list_of_upregulated_and_downregulated_genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of ATAC-seq data and combined ATAC-seq/RNA-seq of Etv2 induced MEF reprogramming and ES/EB differentiation



In [1]:
start_time0 <- Sys.time()

## Install R packages

In [2]:
system('apt-get -q install libgsl-dev', intern = TRUE)

[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "  libgsl23 libgslcblas0"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 [6] "Suggested packages:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
 [7] "  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html"                                                           

In [3]:
system('apt-get install libcairo2-dev', intern = TRUE) # for Cairo package that is required by ComplexHeatmap package

[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "  libcairo-script-interpreter2 libpixman-1-dev libxcb-shm0-dev"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [6] "Suggested packages:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
 [7] "  libcairo2-doc"                                                                                                       

In [4]:
install.packages('BiocManager')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
required_packages <- c(
  'devtools', 'circlize', 'dplyr', 'gplots', 'plyr',
  'SummarizedExperiment', 'BSgenome.Mmusculus.UCSC.mm10', 'BiocParallel', 
  'rtracklayer', 'EnrichedHeatmap', 'ChIPpeakAnno', 'ChIPseeker', 'TxDb.Mmusculus.UCSC.mm10.knownGene', 
  'gsl', 'DirichletMultinomial', 'chromVAR', 'motifmatchr', 'PWMEnrich',
  'ComplexHeatmap',
  'DESeq2',
  'VennDiagram', 'ggwordcloud'
)

In [6]:
missing_packages <- required_packages[!required_packages %in% rownames(installed.packages())]
if (length(missing_packages) > 0){
    BiocManager::install(missing_packages)
}

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'BiocVersion', 'circlize', 'gplots', 'plyr',
  'SummarizedExperiment', 'BSgenome.Mmusculus.UCSC.mm10', 'BiocParallel',
  'rtracklayer', 'EnrichedHeatmap', 'ChIPpeakAnno', 'ChIPseeker',
  'TxDb.Mmusculus.UCSC.mm10.knownGene', 'gsl', 'DirichletMultinomial',
  'chromVAR', 'motifmatchr', 'PWMEnrich', 'ComplexHeatmap', 'DESeq2',
  'VennDiagram', 'ggwordcloud'

also installing the dependencies ‘gridGraphics’, ‘gridExtra’, ‘fastmatch’, ‘tweenr’, ‘polyclip’, ‘ggplotify’, ‘patchwork’, ‘pracma’, ‘R.oo’, ‘R.methodsS3’, ‘formatR’, ‘plogr’, ‘filelock’, ‘DO.db’, ‘fgsea’, ‘qvalue’, ‘ggforce’, ‘ggrepel’, ‘viridis’, ‘tidygraph’, ‘graphlayouts’, ‘ggfun’, ‘GO.db’, ‘ape’, ‘aplot’, ‘tidytree’, ‘treeio’, ‘yulab.utils’, ‘poweRlaw’, ‘R.utils’, ‘sass’, ‘bitops

In [7]:
install.packages('xlsx')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rJava’, ‘xlsxjars’




In [8]:
devtools::install_github("GreenleafLab/chromVARmotifs")

pillar    (1.6.2  -> 1.6.3 ) [CRAN]
stringi   (1.7.4  -> 1.7.5 ) [CRAN]
openssl   (1.4.3  -> 1.4.5 ) [CRAN]
mime      (0.11   -> 0.12  ) [CRAN]
cpp11     (0.3.1  -> 0.4.0 ) [CRAN]
lifecycle (1.0.0  -> 1.0.1 ) [CRAN]
hms       (1.1.0  -> 1.1.1 ) [CRAN]
tibble    (3.1.4  -> 3.1.5 ) [CRAN]
digest    (0.6.27 -> 0.6.28) [CRAN]
readr     (2.0.1  -> 2.0.2 ) [CRAN]


Installing 10 packages: pillar, stringi, openssl, mime, cpp11, lifecycle, hms, tibble, digest, readr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✔  checking for file ‘/tmp/RtmpO1mpMp/remotes59684acf2b/GreenleafLab-chromVARmotifs-38bed55/DESCRIPTION’
─  preparing ‘chromVARmotifs’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘chromVARmotifs_0.2.0.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
library(chromVARmotifs)
library(chromVAR)
library(BSgenome.Mmusculus.UCSC.mm10)
library(motifmatchr)
library(BiocParallel)
library(SummarizedExperiment)
library(gplots)
library(circlize)
library(ComplexHeatmap)
register(MulticoreParam(2)) 
library(plyr)
library(dplyr)
library(DESeq2)
library(VennDiagram)
library(ggwordcloud) 
library(xlsx)



Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attac

## Read replicate-level ATAC-seq data

In [10]:
# Read the ATAC-seq read counts data for each sample and count the motifs at each interval
se <- readRDS(gzcon(url('https://s3.msi.umn.edu/gongx030/datasets/dataset=Etv2ATAC_version=20190228b/ATAC_peaks_Etv2_reprogramming.rds')))
se <- se[, colData(se)$group %in% c('EB_NoDox_D25', 'EB_Dox_D25', 'EB_Dox_D25_Flk1pos', 'MEF_NoDox', 'MEF_Dox_D1', 'MEF_Dox_D2', 'MEF_Dox_D7', 'MEF_Dox_D7_Flk1pos')]
se <- se[rowSums(assays(se)$counts > 0) >= 2]
is_MEF <- colData(se)$group %in% c('MEF_NoDox', 'MEF_Dox_D1', 'MEF_Dox_D2', 'MEF_Dox_D7', 'MEF_Dox_D7_Flk1pos')
is_EB <- colData(se)$group %in% c('EB_NoDox_D25', 'EB_Dox_D25', 'EB_Dox_D25_Flk1pos')
se

class: RangedSummarizedExperiment 
dim: 168305 16 
metadata(0):
assays(1): counts
rownames(168305): chr1 chr1 ... chrY chrY
rowData names(0):
colnames: NULL
colData names(9): study_accession run ... tissue bw_file

In [11]:
colData(se)$group %>% table()

.
        EB_Dox_D25 EB_Dox_D25_Flk1pos       EB_NoDox_D25         MEF_Dox_D1 
                 2                  2                  2                  2 
        MEF_Dox_D2         MEF_Dox_D7 MEF_Dox_D7_Flk1pos          MEF_NoDox 
                 2                  2                  2                  2 

## Read scRNA-seq of MEF reprogramming

In [12]:
sc_file <- 'https://s3.msi.umn.edu/gongx030/etv2_pioneer/data/processed_Etv2_scRNAseq.rds'
command <- sprintf('wget -x -c -nH %s', sc_file)
command %>% system()

In [13]:
se_sc <- gsub('https://s3.msi.umn.edu/', './', sc_file) %>% readRDS()
table(colData(se_sc)$group)


  MEF_NoDox  MEF_Dox_D1  MEF_Dox_D2 MEF_Dox_D7a MEF_Dox_D7b 
       3562         948        2936        7202         827 

## Find the DE genes between MEF D7 Flk1+ cells and undifferentiated MEF

In [14]:
X <- assays(se_sc)$scaled_counts
j_MEF_NoDox <- colData(se_sc)$group %in% 'MEF_NoDox'
j_MEF_Dox_D7_Flk1pos <- colData(se_sc)$group %in% c('MEF_Dox_D7b')
pvalues <- mclapply(
  1:nrow(X), 
  function(i) wilcox.test(X[i, j_MEF_Dox_D7_Flk1pos], X[i, j_MEF_NoDox])$p.value,
  mc.cores = 2
) %>% unlist()

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)



## Read bulk RNA-seq of EB differentiation

In [15]:
se_bulk <- readRDS(gzcon(url('https://s3.msi.umn.edu/gongx030/datasets/dataset=Etv2RNA-seq_version=20190909a/se.rds')))

## Find  DE genes between no dox and Flk1+ cells post induction

In [16]:
se_bulk <- DESeqDataSet(se_bulk, design = ~ group)
se_bulk <- estimateSizeFactors(se_bulk)
se_bulk <- DESeq(se_bulk)
assays(se_bulk)$normalized_counts <- log2(counts(se_bulk, normalized = TRUE) + 1)
res <- results(se_bulk, contrast = c('group', 'EB_Dox_D25_Flk1pos_Etv2', 'EB_NoDox_D25_Etv2'))

Warning message in DESeqDataSet(se_bulk, design = ~group):
“some variables in design formula are characters, converting to factors”
using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Merge the scRNA-seq and bulk RNA-seq by gene symbols.


In [17]:
d_rna_merge <- merge(
  data.frame(
    symbol = rownames(res), 
    EB_log2FoldChange = res$log2FoldChange, 
    EB_pvalue = res$pvalue
  ),
  data.frame(
    symbol = rowData(se_sc)$name, 
    MEF_log2FoldChange = rowMeans(X[, j_MEF_Dox_D7_Flk1pos]) - rowMeans(X[, j_MEF_NoDox]), 
    MEF_pvalue = pvalues
  ),
  by.x = 'symbol',
  by.y = 'symbol'
) %>%
  filter(!is.na(EB_pvalue) & !is.na(MEF_pvalue)) %>%
  filter(!duplicated(symbol))
rownames(d_rna_merge) <- d_rna_merge$symbol
d_rna_merge %>% filter(symbol %in% c('Etv2', 'Kdr', 'Emcn', 'Cdh5', 'Runx1'))

,symbol,EB_log2FoldChange,EB_pvalue,MEF_log2FoldChange,MEF_pvalue
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Cdh5,Cdh5,7.534863,1.315189e-43,1.5647330,0.000000e+00
Emcn,Emcn,5.908320,6.634402e-03,1.4578199,0.000000e+00
Etv2,Etv2,5.462880,0.000000e+00,1.1166636,3.666651e-272
Kdr,Kdr,7.093824,0.000000e+00,1.7277936,0.000000e+00
Runx1,Runx1,1.891086,1.139845e-11,-0.3865211,1.138801e-25


## chromVAR analysis of combined ATAC-seq data


In [18]:
data("mouse_pwms_v2") 
se <- addGCBias(se, genome = BSgenome.Mmusculus.UCSC.mm10)
motif_ix <- matchMotifs(mouse_pwms_v2, se, genome = 'mm10')

### Looking for significantly changed motifs in EB

In [19]:
set.seed(4321)
valid <- rowSums(assays(se[, is_EB])$counts > 0) >= 2
dev_EB <- computeDeviations(object = se[valid, is_EB], annotations = motif_ix[valid, ])
v_EB <- computeVariability(dev_EB)

In [20]:
d_EB_Dox_D25_Flk1pos <- rowMeans(assays(dev_EB)$deviations[, colData(dev_EB)$group %in% c('EB_Dox_D25_Flk1pos')])
d_EB_NoDox_D25 <- rowMeans(assays(dev_EB)$deviations[, colData(dev_EB)$group %in% c('EB_NoDox_D25')])

In [21]:
dat_EB <- assays(dev_EB)$deviations
colnames(dat_EB) <- colData(dev_EB)$group

In [22]:
nms <- unique(colnames(dat_EB))

In [23]:
EB <- as.data.frame(sapply(nms, function(x)  rowMeans(dat_EB[,colnames(dat_EB) %in% x])))

### Looking for significantly changed motifs in MEF

In [24]:
set.seed(8765)
valid <- rowSums(assays(se[, is_MEF])$counts > 0) >= 2
dev_MEF <- computeDeviations(object = se[valid, is_MEF], annotations = motif_ix[valid, ])
v_MEF <- computeVariability(dev_MEF)

In [25]:
d_MEF_Dox_D7_Flk1pos <- rowMeans(assays(dev_MEF)$deviations[, colData(dev_MEF)$group %in% c('MEF_Dox_D7_Flk1pos')])
d_MEF_NoDox <- rowMeans(assays(dev_MEF)$deviations[, colData(dev_MEF)$group %in% c('MEF_NoDox')])

In [26]:
dat_MEF <- assays(dev_MEF)$deviations
colnames(dat_MEF) <- colData(dev_MEF)$group

In [27]:
nms <- unique(colnames(dat_MEF))

In [28]:
MEF <- as.data.frame(sapply(nms, function(x)  rowMeans(dat_MEF[,colnames(dat_MEF) %in% x])))

# Table containing the list of upregulated TFs

In [29]:
up_overlap <- v_EB$p_value_adj < 0.05 & d_EB_Dox_D25_Flk1pos > d_EB_NoDox_D25 & v_MEF$p_value_adj < 0.05 & d_MEF_Dox_D7_Flk1pos > d_MEF_NoDox

In [30]:
up <- v_EB[up_overlap,]
up$p_value_MEF <- v_MEF$p_value[match(up$name, v_MEF$name)]
up$p_value_adj_MEF <- v_MEF$p_value_adj[match(up$name, v_MEF$name)]
up$EB_D25_NoDox <- EB$EB_NoDox_D25[match(rownames(up), rownames(EB))]
up$EB_D25_Dox_Flk1pos <- EB$EB_Dox_D25_Flk1pos[match(rownames(up), rownames(EB))]
up$MEF_NoDox <- MEF$MEF_NoDox[match(rownames(up), rownames(MEF))]
up$MEF_Dox_D7_Flk1pos <- MEF$MEF_Dox_D7_Flk1pos[match(rownames(up), rownames(MEF))]

In [31]:
up_names <- colnames(up)

In [32]:
up_names <- up_names[!grepl('var|bound', up_names)]

In [33]:
write.xlsx(up[,up_names], 'pioneer_supplementary_table4.xlsx', sheetName = 'Sheet1', col.names = TRUE, row.names = TRUE)

#  Table containing the list of downregulated TFs

In [34]:
down_overlap <- v_EB$p_value_adj < 0.05 & d_EB_Dox_D25_Flk1pos < d_EB_NoDox_D25 & v_MEF$p_value_adj < 0.05 & d_MEF_Dox_D7_Flk1pos < d_MEF_NoDox

In [35]:
down <- v_EB[down_overlap,]
down$p_value_MEF <- v_MEF$p_value[match(down$name, v_MEF$name)]
down$p_value_adj_MEF <- v_MEF$p_value_adj[match(down$name, v_MEF$name)]
down$EB_D25_NoDox <- EB$EB_NoDox_D25[match(rownames(down), rownames(EB))]
down$EB_D25_Dox_Flk1pos <- EB$EB_Dox_D25_Flk1pos[match(rownames(down), rownames(EB))]
down$MEF_NoDox <- MEF$MEF_NoDox[match(rownames(down), rownames(MEF))]
down$MEF_Dox_D7_Flk1pos <- MEF$MEF_Dox_D7_Flk1pos[match(rownames(down), rownames(MEF))]


In [36]:
down_names <- colnames(down)

In [37]:
down_names <- down_names[!grepl('var|bound', down_names)]

In [38]:
write.xlsx(down[,down_names], 'pioneer_supplementary_table5.xlsx', sheetName = 'Sheet1', col.names = TRUE, row.names = TRUE)

In [39]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8          LC_NUMERIC=C                 
 [3] LC_TIME=en_US.UTF-8           LC_COLLATE=en_US.UTF-8       
 [5] LC_MONETARY=en_US.UTF-8       LC_MESSAGES=en_US.UTF-8      
 [7] LC_PAPER=en_US.UTF-8          LC_NAME=en_US.UTF-8          
 [9] LC_ADDRESS=en_US.UTF-8        LC_TELEPHONE=en_US.UTF-8     
[11] LC_MEASUREMENT=en_US.UTF-8    LC_IDENTIFICATION=en_US.UTF-8

attached base packages:
 [1] grid      stats4    parallel  stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] xlsx_0.6.5                         ggwordcloud_0.5.0                 
 [3] ggplot2_3.3.5                      VennDiagram_1.6.20                
 [5] futile.logger_1.4.3                DESeq2_1

In [40]:
Sys.time() - start_time0

Time difference of 44.51343 mins